# Naive CAPM Calculations #

### CAPM Model using Simple Linear Regression Model ###

In [64]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Get Data
from fredapi import Fred
import logging

# Pretty Notation
from IPython.display import display, Math

In [65]:
# Data Collection Function from FRED
def get_fred_data(symbol: str) -> pd.DataFrame:
    
    fred_key = '0174cb93931388a2bf305663e4117fd3'

    fred = Fred(api_key = fred_key)
    
    df = fred.get_series(symbol)
    
    return df

In [66]:
# Nvidia Data
df_nvda = pd.read_csv(rf"..\stocks\NVDA.csv")
df_nvda = df_nvda.set_index('Date')
df_nvda.index = pd.to_datetime(df_nvda.index)

df_nvda = df_nvda.loc['2015-01-01':]

df_nvda

In [67]:
# We will use the 10 years bond rate as the RFR

rf = get_fred_data('DGS10')
rf.index = pd.to_datetime(rf.index)
rf = rf.loc['2015-01-01':]

rf

In [68]:
# Nvidia Data
sp500 = pd.read_csv(r"..\additional_data\sp500.csv")
sp500 = sp500.set_index('Date')
sp500.index = pd.to_datetime(sp500.index)

sp500

In [69]:
# Create the DataFrame

data = pd.DataFrame()

data['stock_price'] = df_nvda['Adjusted_close']
data['annual_rfr'] = rf
data['benchmark'] = sp500

data = data.dropna()

data

In [70]:
# Now the returns

data['stock_returns'] = data['stock_price'].pct_change(1).mul(100)
data['daily_rfr'] = (((1 + (data['annual_rfr'].div(100)))**(1/360)) - 1).mul(100)
data['market_returns'] = data['benchmark'].pct_change(1).mul(100)

data

In [71]:
# We use the excess returns

data['stock_excess'] = data['stock_returns'] - data['daily_rfr']
data['market_excess'] = data['market_returns'] - data['daily_rfr']

data

In [72]:
# We know how to calculate a beta, but not a rolling beta

data['rolling_cov'] = data['stock_excess'].rolling(window=252).cov(data['market_excess'])
data['rolling_var_market'] = data['market_excess'].rolling(window=252).var()

# Rolling beta
data['rolling_beta'] = data['rolling_cov'] / data['rolling_var_market']

data.dropna()

In [73]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(data['rolling_beta'], label='Beta', color='blue', alpha=0.7)
plt.axhline(y=data['rolling_beta'].mean(), color='black', linestyle='dashed', label='Beta Mean')

# Config
plt.title('Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Beta')
plt.legend()

# Show
plt.show()

In [74]:
# Now we are interested in obtaining the alpha

data['rolling_avg_stock'] = data['stock_excess'].rolling(window=252).mean()
data['rolling_avg_market'] = data['market_excess'].rolling(window=252).mean()

data['rolling_alpha'] = data['rolling_avg_stock'] - data['rolling_beta'] * data['rolling_avg_market']

data.dropna()

In [75]:
# Create Plot

# Crear el gráfico de la serie de tiempo y las medias móviles exponenciales
plt.figure(figsize=(10, 6))
plt.plot(data['rolling_alpha'], label='Alpha', color='orange', alpha=0.7)
plt.axhline(y=data['rolling_alpha'].mean(), color='black', linestyle='dashed', label='Alpha Mean')

# Añadir título y etiquetas
plt.title('Alpha Time Series')
plt.xlabel('Time')
plt.ylabel('Alpha')
plt.legend()

# Mostrar el gráfico
plt.show()

In [76]:
# Create Figure
fig, ax1 = plt.subplots(dpi = 300)

# Unemployment Rate Plot
data['rolling_beta'].plot(color = 'blue', ax = ax1, alpha=0.7)
ax1.set_xlabel('Date')
ax1.set_ylabel(
    'Beta', 
    color='blue'
    )

# Unemployment Mentions Plot
ax2 = ax1.twinx()

data['rolling_alpha'].plot(color = 'orange', ax = ax2, alpha=0.7)
ax2.set_ylabel(
    'Alpha', 
    color='orange'
    )

plt.title('Beta and Alpha Time Series')
plt.show()

In [77]:
# Security Market Line

# Parámeters
risk_free_rate = data['daily_rfr'].iloc[-1]
market_risk_premium = data['rolling_avg_market'].iloc[-1]

# Random Betas
betas = np.linspace(0, 3, 100)  # Rango de betas de 0 a 2

# Expected Returns
expected_returns = risk_free_rate + market_risk_premium * betas

# Plot
plt.figure(figsize=(10, 6))

plt.plot(betas, expected_returns, color='black', linestyle='--', label='Security Market Line')
plt.scatter(data['rolling_beta'].iloc[-1], data['rolling_avg_stock'].iloc[-1], color='red', s=50, label='expected returns')  

# Config
plt.title('Security Market Line (SML)')
plt.xlabel('Beta')
plt.ylabel('Expected Return')

# RFR Line
plt.axhline(y=risk_free_rate, color='grey', linestyle='--', label='Risk-Free Rate')

# Legends
plt.legend()

plt.grid(True)
plt.show()

In [78]:
# Now we have to calculate the Treynor's Ratio

data['treynors_ratio'] = data['stock_excess'].div(data['rolling_beta'])

data.dropna()

In [79]:
# And the Sharpe Ratio

data['rolling_std_stock'] = data['stock_returns'].rolling(window=252).std()
data['sharpe_ratio'] = data['stock_excess'].div(data['rolling_std_stock'])

data.dropna()

In [80]:
# Create Plot to demonstrate the Sharpe Ratio is always smaller than the Treynors Ratio

plt.figure(figsize=(10, 6))
plt.plot(abs(data['treynors_ratio']), label='Treynors Ratio', color='green', alpha=0.7)
plt.plot(abs(data['sharpe_ratio']), label='Sharpe Ratio', color='red', alpha=0.7)

# Config
plt.title('Ratios Time Series')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()

# Show
plt.show()

In [81]:
plt.figure(figsize=(10, 6))
plt.plot(abs(data['treynors_ratio'] / abs(data['sharpe_ratio'])), label='Ratios Difference', color='green', alpha=0.7)
plt.axhline(y=1, color='grey', linestyle='--')

# Config
plt.title('Ratios Time Series')
plt.xlabel('Time')
plt.ylabel('Values')
plt.legend()

# Show
plt.show()

In [82]:
data['rolling_beta'].dropna()